# Video Classification with Transformers

In [10]:
from google.colab import drive
drive.mount('/content/drive')
# !wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
# !tar xf ucf101_top5.tar.gz
!pip install -q git+https://github.com/tensorflow/docs
# !pip install -q py-feat
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from google.colab.patches import cv2_imshow
from sklearn.model_selection import KFold
from tensorflow import keras
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
# from feat import Detector
import pandas as pd
import numpy as np
import imageio
import cv2
import os

# face_model = "RetinaFace"
# landmark_model = "MobileNet"
# au_model = "svm"
# emotion_model = "resmasknet" #resmasknet,fer, svm, rf
# detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

MAX_SEQ_LENGTH = 300
NUM_FEATURES = 1024
IMG_SIZE = 224
EPOCHS = 100

# test_df['tag'] = test_df['tag'].replace({'stress':'angry'})
# from google.colab import files
# test_df.to_csv('test_df_5class.csv') 
# files.download('test_df_5class.csv')
# ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/test_df_5class.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# train_df = pd.read_csv("drive/MyDrive/seqimg_youtube/unzip/labels.csv")
# train_df['tag'] = train_df['tag'].replace({'stress':'angry'})
# pd.unique(train_df['tag'])
# from google.colab import files
# train_df.to_csv('labels.csv') 
# files.download('labels.csv')

In [12]:
train_df = pd.read_csv("drive/MyDrive/seqimg_youtube/unzip/labels_5classes.csv")
# test_df = pd.read_csv("drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/test_df_5class.csv")
print(f"Total videos for training: {len(train_df)}")
# print(f"Total videos for testing: {len(test_df)}")
center_crop_layer = layers.experimental.preprocessing.CenterCrop(IMG_SIZE, IMG_SIZE)

# def crop_center(frame):
#     faceimg = detector.detect_faces(frame)
#     try:
#       pos = faceimg[0][:4]
#       cropimg =frame[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] 
#       cropped = cv2.resize(cropimg, (224, 224), interpolation=cv2.INTER_CUBIC)
#     except:
#       cropped = frame
#       cropped = cv2.resize(cropped, (224, 224), interpolation = cv2.INTER_AREA)
#       print('error')
#     return cropped

def load_video(path, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # frame = crop_center(frame)
            frame = cv2.resize(frame, (224,224))
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(weights="imagenet",include_top=False,pooling="avg",input_shape=(IMG_SIZE, IMG_SIZE, 3),)
    preprocess_input = keras.applications.densenet.preprocess_input
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)
    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")
feature_extractor = build_feature_extractor()
label_processor = keras.layers.experimental.preprocessing.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None)
print(label_processor.get_vocabulary())

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].tolist()
    labels = df["tag"]
    labels = label_processor(labels[..., None]).numpy()
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for idx, path in tqdm(enumerate(video_paths)):
        frames = load_video(os.path.join(root_dir, path))
        print('idx', idx, 'path', path)
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            # print('frames', len(frames))
            # print('diff=', diff)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            # print('padding=', padding.shape)
            # print('frames =', frames.shape)
            frames = np.concatenate((frames, padding),axis=0)
            # print('frames=', frames.shape)
        frames = frames[None, ...]
        temp_frame_featutes = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")
        # print('frames', frames.shape)

        for i, batch in enumerate(frames):
            # print('batch', batch.shape)
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            # print('video_length', video_length)
            # print('###########')
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_featutes[i, j, :] = feature_extractor.predict(batch[None, j, :])
                else:
                    temp_frame_featutes[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_featutes.squeeze()
        # np.save('drive/MyDrive/seqimg_youtube/video_transformers_croped/frame_features'+str(idx)+'.npy', frame_features)
        # np.save('drive/MyDrive/seqimg_youtube/video_transformers_croped/labels'+str(idx)+'.npy', labels)
    return frame_features, labels

Total videos for training: 1394
['angry', 'happy', 'neutral', 'relax', 'sad']


In [13]:
# np.save('frame_start275.npy', frame_features)
# np.save('frame_label275.npy', labels)

In [ ]:
# !wget -q https://git.io/JZmf4 -O top5_data_prepared.tar.gz
# !tar xf top5_data_prepared.tar.gz

# A = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/labels343.npy')
# A[343]
# ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features342.npy'
# A = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features342.npy')
# A[343]
# video_paths = train_df["video_name"].values[343:].tolist()
# for idx, path in tqdm(enumerate(video_paths)):
#   print(idx)
# for i in range(1194):
#   A = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features'+str(i)+'.npy')
#   print('i=' , i, ' value = ' , A[i-1].sum())

# np.save('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/test_data.npy', frame_features)
# np.save('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/test_labels.npy', labels)
frame_features, labels = prepare_all_videos(train_df, 'drive/MyDrive/seqimg_youtube/unzip/cropped_v1') #343 , 937   
# ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/'
# ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/labels_total.npy'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


idx 0 path face2_01_0_59_3296_neutral.mp4
idx 1 path face2_01_1_3356_4075_relax.mp4
idx 2 path face2_01_2_4135_5034_neutral.mp4
idx 3 path face2_01_3_5514_5754_stress.mp4
idx 4 path face2_01_4_5814_6053_stress.mp4
idx 5 path face2_01_5_6053_6233_happy.mp4
idx 6 path face2_01_6_6293_7012_neutral.mp4
idx 7 path face2_01_7_7072_7252_stress.mp4
idx 8 path face2_01_8_7312_7672_neutral.mp4
idx 9 path face2_01_9_7732_8031_stress.mp4
idx 10 path face2_01_10_8091_8451_happy.mp4
idx 11 path face2_01_11_8451_8871_happy.mp4
idx 12 path face2_01_12_9050_9230_happy.mp4
idx 13 path face2_01_13_9410_9650_happy.mp4
idx 14 path face2_01_14_9710_10129_neutral.mp4
idx 15 path face2_01_15_11448_11568_stress.mp4
idx 16 path face2_01_16_11628_11928_neutral.mp4
idx 17 path face2_01_17_10909_12647_stress.mp4
idx 18 path face2_01_18_12707_12887_sad.mp4
idx 19 path face2_01_19_12947_14565_sad.mp4
idx 20 path face2_01_20_14625_14685_sad.mp4
idx 21 path face2_01_21_14745_14925_angry.mp4
idx 22 path face2_01_22_144

In [9]:
np.save('drive/MyDrive/seqimg_youtube/frame_features.npy', frame_features)
np.save('drive/MyDrive/seqimg_youtube/labels.npy', labels)

In [ ]:
# A = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features342.npy')
# B = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features936.npy')
# C = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features1193.npy')

# X1_new = A[:343]
# # print(A[342])
# X2_new = B[343:937]
# # print(B[936])
# X3_new = C[937:1194]
# # print(C[1193])

# print('total', X1_new.shape[0]+X2_new.shape[0]+X3_new.shape[0])
# print('X1_new', X1_new.shape)
# print('X2_new', X2_new.shape)
# print('X3_new', X3_new.shape)
# Xtotal = np.concatenate((X1_new, X2_new, X3_new), axis=0)

# labelss = np.load('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/test_labels0.npy')
# np.save('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/labels_total.npy', labelss)
# print('Xtotal', labelss.shape)

In [ ]:
train_data  = np.load("drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/frame_features_Xtotal.npy")
train_labels = np.load("drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/video_transformers_croped/labels_total.npy")
# test_data, test_labels = np.load('/content/test_data.npy'), np.load('/content/test_labels.npy')
print(f"Frame features in train set: {train_data.shape}")

Frame features in train set: (1194, 20, 1024)


## Building the Transformer-based model

We will be building on top of the code shared in
[this book chapter](https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-11) of
[Deep Learning with Python (Second ed.)](https://www.manning.com/books/deep-learning-with-python)
by François Chollet.

First, self-attention layers that form the basic blocks of a Transformer are
order-agnostic. Since videos are ordered sequences of frames, we need our
Transformer model to take into account order information.
We do this via **positional encoding**.
We simply embed the positions of the frames present inside videos with an
[`Embedding` layer](https://keras.io/api/layers/core_layers/embedding). We then
add these positional embeddings to the precomputed CNN feature maps.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

# Now, we can create a subclassed layer for the Transformer.
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

## Utility functions for training
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    classes = len(label_processor.get_vocabulary())
    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(sequence_length, embed_dim, name="frame_position_embedding")(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

def run_experiment():
    filepath = "tmp1/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, save_weights_only=True, save_best_only=False, verbose=1)
    model = get_compiled_model()
    history = model.fit(train_data,train_labels,validation_split=0.15,epochs=30,callbacks=[checkpoint],)
    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    return model

## Model training and inference

In [ ]:
# trained_model = run_experiment()

num_folds = 10
acc_per_fold = []
loss_per_fold = []
fold_no = 1

kfold = KFold(n_splits=num_folds, shuffle=True)
for train, test in kfold.split(train_data, train_labels):
    model = get_compiled_model()
    history = model.fit(train_data[train], train_labels[train],epochs=100)
    _, accuracy = model.evaluate(train_data[test], train_labels[test])
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    acc_per_fold.append(round(accuracy * 100, 2))
    print(f'Training for fold {fold_no} ...')
    fold_no = fold_no + 1

print('mean = ', np.mean(acc_per_fold))

Epoch 1/100
34/34 [==============================] - 5s 12ms/step - loss: 3.4228 - accuracy: 0.2458
Epoch 2/100
34/34 [==============================] - 0s 11ms/step - loss: 2.1006 - accuracy: 0.2598
Epoch 3/100
34/34 [==============================] - 0s 11ms/step - loss: 1.9251 - accuracy: 0.2663
Epoch 4/100
34/34 [==============================] - 0s 12ms/step - loss: 1.8836 - accuracy: 0.2477
Epoch 5/100
34/34 [==============================] - 0s 12ms/step - loss: 1.7703 - accuracy: 0.2747
Epoch 6/100
34/34 [==============================] - 0s 11ms/step - loss: 1.6577 - accuracy: 0.2868
Epoch 7/100
34/34 [==============================] - 0s 11ms/step - loss: 1.5825 - accuracy: 0.2924
Epoch 8/100
34/34 [==============================] - 0s 11ms/step - loss: 1.4730 - accuracy: 0.3501
Epoch 9/100
34/34 [==============================] - 0s 12ms/step - loss: 1.4241 - accuracy: 0.3641
Epoch 10/100
34/34 [==============================] - 0s 11ms/step - loss: 1.4226 - accuracy: 0.4004

In [ ]:
def prepare_single_video(frames):
    frame_featutes = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")
    # Pad shorter videos.
    if len(frames) < MAX_SEQ_LENGTH:
        diff = MAX_SEQ_LENGTH - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        print(diff,padding )
        frames = np.concatenate(frames, padding)
    frames = frames[None, ...]

    # Extract features from the frames of the current video.
    for i, batch in enumerate(frames):
        video_length = batch.shape[1]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_featutes[i, j, :] = feature_extractor.predict(batch[None, j, :])
            else:
                frame_featutes[i, j, :] = 0.0
    return frame_featutes


def predict_action(path):
    class_vocab = label_processor.get_vocabulary()
    frames = load_video(os.path.join("drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/test", path))
    print(os.path.join("drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/season2/file_v1/csv_frame/trainv1/videos/test", path))
    frame_features = prepare_single_video(frames)
    probabilities = trained_model.predict(frame_features)[0]
    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = predict_action(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])